# 3. Régressions linéaires

Maintenant que nous avons décrit les variables ainsi que leurs relations entre elles. Il est intéressant de faire une régression afin de fournir un début d'explication sur l'effet des variables socio-démographiques sur la dispersion des trotinettes. Nous allons ainsi faire une régression afin d'expliquer le nombre de trottinettes dans chaque quartiers admnistratifs de Paris. En ouverture, nous allons nous intérésser aussi à la relation entre le nombre de trottinettes et la distance aux moyens de transports.

## 3.1 Préparation des données

Les base telles qu'elles sont ne permettent pas de pouvoir faire des régressions, notamment parce que des variables sont qualitatives. De plus, il faut pour pouvoir utiliser la distance aux moyens de transport créer différentes catégories en fonction de leurs distances.

In [8]:
path="/Users/dalilyoucefi/Documents/ProjetInfo/"

In [9]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
import geopandas as gpd


In [10]:
df = pd.read_csv(path+'BaseQuartierLuSa.csv')
gdf = gpd.read_file(path+'GeoTierLunsSA.geojson')

On regarde comment se répartit la distance aux moyens de transport

In [11]:
gdf["min_distance"].min()

0.002546362541187594

In [12]:
gdf["min_distance"].quantile(0.5)

0.20971384983506855

In [13]:
gdf["min_distance"]

0        0.238695
1        0.085875
2        0.086848
3        0.051079
4        0.335544
           ...   
32067    0.194821
32068    0.281159
32069    0.281159
32070    0.297171
32071    0.297171
Name: min_distance, Length: 32072, dtype: float64

On crée plusieurs tranches de distance aux moyens de transport

In [14]:
li=[]

for i in range(len(gdf)):
    if gdf["min_distance"][i]<0.05:
        li.append("0-50")
    if gdf["min_distance"][i]>=0.05 and gdf["min_distance"][i]<0.1:
        li.append("50-100")
    if gdf["min_distance"][i]>=0.1 and gdf["min_distance"][i]<0.15:
        li.append("100-150")
    if gdf["min_distance"][i]>=0.15 and gdf["min_distance"][i]<0.2:
        li.append("150-200")
    if gdf["min_distance"][i]>=0.2 and gdf["min_distance"][i]<0.25:
        li.append("200-250")
    if gdf["min_distance"][i]>=0.25 and gdf["min_distance"][i]<0.3:
        li.append("250-300")
    if gdf["min_distance"][i]>=0.3 and gdf["min_distance"][i]<0.35:
        li.append("300-350")
    if gdf["min_distance"][i]>=0.35 and gdf["min_distance"][i]<0.4:
        li.append("350-400")
    if gdf["min_distance"][i]>=0.4 and gdf["min_distance"][i]<0.45:
        li.append("400-450")
    if gdf["min_distance"][i]>=0.45 and gdf["min_distance"][i]<0.5:
        li.append("450-500")
    if gdf["min_distance"][i]>=0.5:
        li.append("500+")

In [15]:
gdf["dist_transport"]=li
gdf['dist_transport']

0        200-250
1         50-100
2         50-100
3         50-100
4        300-350
          ...   
32067    150-200
32068    250-300
32069    250-300
32070    250-300
32071    250-300
Name: dist_transport, Length: 32072, dtype: object

In [16]:
gdf.min_distance

0        0.238695
1        0.085875
2        0.086848
3        0.051079
4        0.335544
           ...   
32067    0.194821
32068    0.281159
32069    0.281159
32070    0.297171
32071    0.297171
Name: min_distance, Length: 32072, dtype: float64

In [17]:
gdf.dist_transport

0        200-250
1         50-100
2         50-100
3         50-100
4        300-350
          ...   
32067    150-200
32068    250-300
32069    250-300
32070    250-300
32071    250-300
Name: dist_transport, Length: 32072, dtype: object

On compte le nombre de trotinnette en fonction du quartier où elles se trouvent, du jour et de l'heure et de la distance au moyen de transport.

In [18]:
gdf2=gdf[["ID","dist_transport","id_quartier","Heure","Jour"]].groupby(["dist_transport","id_quartier","Heure","Jour"],as_index=False).count()


In [19]:
gdf2

,dist_transport,id_quartier,Heure,Jour,ID
0,0-50,1,14h,Lundi,3
1,0-50,1,19h,Lundi,1
2,0-50,1,19h,Samedi,1
3,0-50,1,9h,Lundi,3
4,0-50,1,9h,Samedi,1
...,...,...,...,...,...
4876,500+,79,18h,Samedi,2
4877,500+,79,19h,Lundi,2
4878,500+,79,19h,Samedi,2
4879,500+,79,9h,Lundi,3


In [20]:
df.columns

Index(['Unnamed: 0', 'Jour', 'c_qu', 'l_qu', 'prop_0-14', 'prop_15-29',
       'prop_30-44', 'prop_45-59', 'prop_60-74', 'prop_75+', 'nombre_0-14',
       'nombre_15-29', 'nombre_30-44', 'nombre_45-59', 'nombre_60-74',
       'nombre_75+', 'pop', 'densite', 'nombretransport', 'numquartier', 'ref',
       'nb_logmt_total', 'nombre_de_commerce', 'nombreent', 'CA 1',
       'nombretrottotal'],
      dtype='object')

On garde 

In [21]:
 for i in range(len(df)):
   
    if df["Jour"][i]=="Lundi":
        df["Jour"]=0
    else:
        df["Jour"]=1


In [22]:
df["numquartier"]=df["numquartier"].astype(str)


In [23]:
df2=df[[ "numquartier", 
        'nombre_0-14',
       'nombre_15-29', 'nombre_30-44', 'nombre_45-59', 'nombre_60-74', 'nombre_75+',
       'nombretransport', 'ref', 'nb_logmt_total',
       'nombre_de_commerce', 'nombreent', 'CA 1',"densite"]]

In [24]:
gdf2=gdf2.merge(df2, left_on="id_quartier",right_on="numquartier")

In [25]:
for i in range (len(gdf2)): 
    gdf2["Heure"][i]=gdf2["Heure"][i].replace('h','')
for i in range(len(df)):
   
    if gdf2["Jour"][i]=="Lundi":
        gdf2["Jour"]=0
    else:
        gdf2["Jour"]=1
gdf2["Heure"]=gdf2["Heure"].astype(int)
       

/Users/dalilyoucefi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
gdf2['dist_transport'] = gdf2['dist_transport'].astype('category')
gdf2['dist_transport'] = gdf2['dist_transport'].cat.reorder_categories(['0-50', '100-150', '150-200', '200-250', '250-300', '300-350',
       '350-400', '400-450', '450-500', '50-100', '500+'], ordered=True)
gdf2['dist_transport'] = gdf2['dist_transport'].cat.codes


## 3.2 Régressions

*nombretrottotal* est la variable expliquée

In [27]:
Y=pd.DataFrame(df["nombretrottotal"])

#### 3.2.1 Premier bloc de la régression

In [28]:
# On effectue une régression de nombretrottotal sur le premier bloc
X1=df[['nombreent','nombre_de_commerce','CA 1']]
X1 = sm.add_constant(X1)
model = sm.OLS(Y,X1)
results1 = model.fit()
print(results1.params)
print(results1.rsquared_adj)

const                 9.228215e+01
nombreent             6.173551e-02
nombre_de_commerce    2.515857e+00
CA 1                 -1.215042e-07
dtype: float64
0.1585283705229773


In [29]:
# Faisons la même chose sur le premier bloc retiré de CA 1
X2=df[['nombreent','nombre_de_commerce']]
X2 = sm.add_constant(X2)
model = sm.OLS(Y,X2)
results2 = model.fit()
print(results2.params)
print(results2.rsquared_adj)

const                 90.851258
nombreent              0.061427
nombre_de_commerce     2.531771
dtype: float64
0.16351680925475154


On observe que le R² ajusté est plus élevé lorsque le premier bloc n'est pas composé de la variable *CA 1*. Comme montré dans la partie 2, la variable *CA 1* est corrélée avec *nombreent*. Donc inclure *CA 1* crée un biais de variable incluse. On va donc la retirer de notre modèle.

#### 3.2.2 Deuxième bloc de la régression

In [30]:
X3=df[['ref','nb_logmt_total']]
X3=sm.add_constant(X3)
model = sm.OLS(Y,X3)
results3 = model.fit()
print(results3.params)
print(results3.rsquared_adj)

const            -501.095355
ref                24.559239
nb_logmt_total      0.051139
dtype: float64
0.18107687824331753


In [31]:
X4=df[['ref']]
X4=sm.add_constant(X4)
model = sm.OLS(Y,X4)
results4 = model.fit()
print(results4.params)
print(results4.rsquared_adj)

const    165.662455
ref        1.352168
dtype: float64
-0.005689631036355669


In [32]:
X5=df[['nb_logmt_total']]
X5=sm.add_constant(X5)
model = sm.OLS(Y,X5)
results5 = model.fit()
print(results5.params)
print(results5.rsquared_adj)

const             167.281242
nb_logmt_total      0.024335
dtype: float64
0.07978743528225618


On avait montré dans la partie 2 que la variable *ref* n'était pas une variable omise si on la retirait du modèle. Or ici les R² ajusté montre que la variable *nb_logmt_total* explique le mieux la variation de la variable *nombretrottotal*. On va donc uniquement conserver dans notre 2ème bloc la variable *nb_logmt_total* 

#### 3.2.3 Troisième bloc de la régression

In [33]:
X6=df[['nombretransport']]
X6=sm.add_constant(X6)
model = sm.OLS(Y,X6)
results6 = model.fit()
print(results6.params)
print(results6.rsquared_adj)


const              129.494232
nombretransport     13.387881
dtype: float64
0.10657271658678136


La variable *nombredetransport* explique 10% de la variation de la variable *nombretrottotal*. On va donc conserver cette variable.

#### 3.2.4 Quatrième bloc de la régression

In [34]:
X7 = df[['densite','nombre_75+']]
X7=sm.add_constant(X7)
model = sm.OLS(Y,X7)
results7 = model.fit()
print(results7.params)
print(results7.rsquared_adj)

const         154.837498
densite        -0.000325
nombre_75+      0.006798
dtype: float64
0.11619499699394564


Le R² ajusté est de 11.6%. On va donc conserver ce bloc

#### 3.2.5 Régression générale

In [35]:
X8=df[['nombreent','nombre_de_commerce','nb_logmt_total','nombretransport','densite','nombre_75+']]
X8=sm.add_constant(X8)
model = sm.OLS(Y,X8)
results8 = model.fit()
print(results8.params)
print(results8.rsquared_adj)

const                 70.470308
nombreent              0.049561
nombre_de_commerce     2.061789
nb_logmt_total         0.007408
nombretransport        6.531985
densite               -0.000468
nombre_75+             0.001638
dtype: float64
0.26407576498580276


On en conclut que toutes choses égales par ailleurs, en moyenne *nombre_de_commerce* augmente le nombre de trottinettes dans un quartier administratif de 2,06. De même toutes choses égales par ailleurs, en moyenne *nombretransport* augmente le nombre de trottinettes par quartier administratif de 6,5. Les autres variables ont très peu d'influence sur la variation du nombre de trottinettes.

#### 3.2.6 Test de significativité statistique

Pour les tests suivants on va considérer les hypothèses suivantes:\
*H0* = Toutes les variables n'ont pas d'effet causal sur *nombretrottotal* \
*H1* = Au moins une variable a un effet causal sur *nombretrottotal* \
On va donc effectuer un test de Fisher pour chaque bloc de notre régression précédente

In [36]:
#Test de Fisher sur le bloc *nombreent* et *nombre_de_commerce*
R1 = [[0,1,0,0,0,0,0],[0,0,1,0,0,0,0]] # matrice R 
fres1 = results8.f_test(R1)
dir(fres1)
print(fres1.fvalue) #valeur du F de Fisher
print(fres1.pvalue) #valeur de la p-value

[[10.68426104]]
4.535089215436615e-05


On obtient une *p-value* très faible. Ainsi on peut rejeter l'hypothèse que le bloc 1 n'a pas déffet causal au seuil de 1%. Ainsi au moins une des deux variables du bloc est statistiquement significative au seuil de 1%.

In [37]:
# Test de Fisher / t-test sur la variable nb_logmt_total
R2 = [0,0,0,1,0,0,0]
fres2= results8.f_test(R2)
dir(fres2)
print(fres2.fvalue) #valeur du t de Student
print(fres2.pvalue) #valeur de la p-value

[[0.99180115]]
0.3208749856617584


On obtient une valeur de *t-Student* très faible. Ainsi on ne peut rejeter l'hypothèse nulle au seuil de 10%. La variable *nb_logmt_total* n'est pas statistiquement significative au seuil de 10%. 

In [38]:
# Test de Fisher / t-test sur la variable nombretransport
R3 = [0,0,0,0,1,0,0]
fres3 = results8.f_test(R3)
dir(fres3)
print(fres3.fvalue) #valeur du t de Student
print(fres3.pvalue) #valeur de la p-value

[[3.64138203]]
0.058233032147806674


On trouve une p-value de 0.58 donc la variable est significative au seuil 10% mais pas au seuil 5% ni de 1%.

In [39]:
# Test de Fisher sur les variables densite et nombre_75+
R4 = [[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]]
fres4 = results8.f_test(R4)
dir(fres4)
print(fres4.fvalue)
print(fres4.pvalue)

[[6.54590836]]
0.0018717784557674441


Le bloc des deux variables a une *p-value* inférieure à 0.01. Ainsi on peut rejeter l'hypothèse *H0* que le bloc n'a pas d'effet causal au seuil de 1%. Ainsi ce bloc est statistiquement significatif au seuil de 1%.

In [40]:
# Test de Fisher sur la régression générale
R5 = [[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]]
fres5 = results8.f_test(R5)
dir(fres5)
print(fres5.fvalue)
print(fres5.pvalue)

[[10.50914162]]
9.4485449543018e-10


On obtient une *p-value* très faible. Ainsi on en conclut qu'au moins un des blocs définis auparavant a un effet causal sur *nombrtrottotal*. 

#### 3.2.7 Ouverture

Il est intéressant aussi de voir comment la distance à un moyen de transport joue dans le positionnement des trotinnettes. Grâce à la variable *dist_transport* que nous avons crée nous allons pouvoir évaluercet effet grâce à une régression à l'aide la base *gdf2*.

In [41]:
Y = gdf2['ID'] # Variable expliquée
X = gdf2[['nombreent','nombre_de_commerce','nb_logmt_total','dist_transport','densite','nombre_75+',"nombretransport"]] #variables explicatives

In [42]:
X=sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.params)
print(results.rsquared_adj)

const                 5.481753
nombreent             0.000696
nombre_de_commerce    0.052991
nb_logmt_total        0.000058
dist_transport       -0.294692
densite              -0.000015
nombre_75+           -0.000056
nombretransport       0.225116
dtype: float64
0.11127538325974495


On trouve ici un R² ajusté de 11% environ, ce qui est beaucoup plus faible que dans les régressions précédentes, aussi parce que le l'échantillon de données est significativement plus grand. On remarque aussi, que seul les variables liés aux transports ont une significativité pratique. Cela peut s'expliquer par le fait qu'on ne travaille plus à un niveau agrégé.\
Testons la significativité de *dist_transport* et *nombretransport* qui sont les variables qui semblent avoir ici le plus d'importance sur la position des trotinettes.

In [43]:
results.pvalues # p-values des coefficients

const                 5.044975e-246
nombreent              2.002487e-08
nombre_de_commerce     2.965012e-61
nb_logmt_total         1.380282e-01
dist_transport         2.164150e-58
densite                3.956640e-69
nombre_75+             1.396606e-06
nombretransport        2.948213e-31
dtype: float64

On trouve pour tous les deux des p-value quasi-nulles  donc ces coefficients sont significatifs.

## Conclusion

Notre analyse nous conduit à conclure  que le nombre de trottinettes présentes dans les quartiers de Paris dépend surtout de la variable *nombretransport*. On remarque aussi que les trotinettes semblent être plus réparties proche des moyens de transports. Cela nous laisse penser que les trottinettes en libre-service se présentent comme une véritable alternative aux moyens de transports. Toutefois, le manque de  certaines données plus précises (à l'échelle de l'arrondissement pour les données démographiques) et le faible échantillon (80 quartiers admnistratifs) limite l'interprétation notamment de la significativité statistiques des coefficients. Pour étendre l'analyse, on pourrait étendre l'étude à d'autres villes pour voir si les mêmes tendances se dégagent.